In [2]:
import pandas as pd
import numpy as np

In [62]:
final = pd.DataFrame(columns=["State","District","Crop","Season","Prediction"])

In [17]:
AN_book = pd.read_csv("AN.csv")
AN_book

,State,District,Crop,Season,2010-11,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20,2020-21
0,AN,NORTH AND MIDDLE ANDAMAN,Arhar/Tur,Rabi,0.308319,0.301905,0.285714,3.000000,0.992806,0.600000,0.046154,0.500000,NaN,NaN,NaN
1,AN,SOUTH ANDAMANS,Arhar/Tur,Rabi,0.643902,NaN,NaN,0.400000,0.400000,0.400000,NaN,NaN,NaN,NaN,NaN
2,AN,NICOBARS,Banana,Whole Year,5.775717,5.995165,8.439834,7.650000,9.208211,3.713733,2.200147,0.058228,0.103015,NaN,NaN
3,AN,NORTH AND MIDDLE ANDAMAN,Banana,Whole Year,12.127854,13.321114,10.031739,6.895274,7.980769,9.490662,5.164032,8.453603,10.759431,NaN,NaN
4,AN,SOUTH ANDAMANS,Banana,Whole Year,15.325000,15.138705,15.132100,11.440789,11.309262,14.218750,14.897211,19.596364,21.446205,NaN,NaN
5,AN,NICOBARS,Coconut,Whole Year,4896.978022,4777.244688,6129.692833,6564.312521,6494.922647,6439.084219,6482.479784,NaN,NaN,NaN,NaN
6,AN,NORTH AND MIDDLE ANDAMAN,Coconut,Whole Year,3462.377317,4252.998909,4843.537415,4559.023066,4734.693878,5306.122449,5442.176871,NaN,NaN,NaN,NaN
7,AN,SOUTH ANDAMANS,Coconut,Whole Year,3107.344633,5561.829475,4929.577465,4494.382022,4800.673779,4938.271605,4713.804714,NaN,NaN,NaN,NaN
8,AN,NICOBARS,Maize,Rabi,4.744792,0.675000,NaN,0.525773,9.166667,1.000000,NaN,NaN,NaN,NaN,NaN
9,AN,NORTH AND MIDDLE ANDAMAN,Maize,Rabi,1.111880,1.500000,1.404082,2.335544,4.725389,3.692308,2.359596,0.574135,2.163532,NaN,NaN


In [29]:
abs(AN_book.iloc[0][4:].ewm(alpha=0.95, adjust=False).mean()-AN_book.iloc[0][4:]).mean()

0.038108699783520855

In [31]:
AN_book.iloc[2][4:].ewm(alpha=0.95, adjust=False).mean()[0:]

2010-11    5.775717
2011-12    5.984193
2012-13    8.317052
2013-14    7.683353
2014-15    9.131968
2015-16    3.984645
2016-17    2.289372
2017-18    0.169785
2018-19    0.106354
2019-20    0.106354
2020-21    0.106354
Name: 2, dtype: float64

In [56]:
def optium_alpha(book, row_index):
    min_MAD = 10000000.0
    opt_alph = -1
    for z in range(10,95,5):
        alph = z/100.0
        MAD = 0.0
        exp_predict = book.iloc[row_index][4:].ewm(alpha=alph,adjust=False).mean()
        for i in range(0,10):
            if pd.isna(book.iloc[row_index][4+1+i]) or pd.isna(exp_predict[i]):
                continue
            
            MAD = MAD + abs(exp_predict[i]-book.iloc[row_index][4+i+1])
            #print(alph, exp_predict[i], book.iloc[row_index][4+i+1],MAD)
        #print(alph,MAD)
        if MAD < min_MAD :
            min_MAD = MAD
            opt_alph = alph
    return opt_alph

In [60]:
def write_prediction(book):
    for i in range(len(book)):
        alph = optium_alpha(book,i)
        z = len(final)
        final.loc[z] = ""
        for j in range(4):
            final.iloc[z][j]=book.iloc[i][j]
        final.iloc[z][4] = book.iloc[i][4:].ewm(alpha=alph, adjust=False).mean()[-1]
        #print(final.iloc[z])


In [63]:
states = ['AN','AP','AR','AS','BR','CT','GA','GJ','HR','HP','JH','KA','KL','MP','MH','MN','ML','MZ','NL','OD','PB','RJ','SK','TN','TG','TR','UK','UP','WB','DD','DN','DL','JK','PY']
for st in states:
    print(st)
    filename = st+".csv"
    book = pd.read_csv(filename)
    write_prediction(book)
#write_prediction(pd.read_csv("AR.csv"))
print(final)

AN
AP
AR
AS
BR
CT
GA
GJ
HR
HP
JH
KA
KL
MP
MH
MN
ML
MZ
NL
OD
PB
RJ
SK
TN
TG
TR
UK
UP
WB
DD
DN
DL
JK
PY
      State                  District       Crop      Season Prediction
0        AN  NORTH AND MIDDLE ANDAMAN  Arhar/Tur        Rabi   0.552343
1        AN            SOUTH ANDAMANS  Arhar/Tur        Rabi   0.400003
2        AN                  NICOBARS     Banana  Whole Year   0.122004
3        AN  NORTH AND MIDDLE ANDAMAN     Banana  Whole Year    9.04364
4        AN            SOUTH ANDAMANS     Banana  Whole Year    21.2133
...     ...                       ...        ...         ...        ...
18009    PY               PONDICHERRY    Tobacco        Rabi    20.3366
18010    PY               PONDICHERRY   Turmeric      Kharif    2.60985
18011    PY                  KARAIKAL       Urad      Summer   0.274382
18012    PY               PONDICHERRY       Urad      Kharif   0.846846
18013    PY                     YANAM       Urad        Rabi   0.776476

[18014 rows x 5 columns]


In [64]:
write_prediction(pd.read_csv("CH.csv"))

In [66]:
final.to_csv("prediction.csv",index=False)